In [4]:
import torch
from diffusers import DiffusionPipeline
from hy3dshape.pipelines import Hunyuan3DDiTFlowMatchingPipeline
from PIL import Image, ImageOps
import os
import warnings

warnings.filterwarnings("ignore")

# --- 1. Инициализация моделей ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

shape_pipeline = Hunyuan3DDiTFlowMatchingPipeline.from_pretrained(
    'tencent/Hunyuan3D-2.1',
    torch_dtype=torch.float16
)

image_gen_pipeline = DiffusionPipeline.from_pretrained(
    "playgroundai/playground-v2.5-1024px-aesthetic",
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")


# --- НОВАЯ ВСПОМОГАТЕЛЬНАЯ ФУНКЦИЯ ---
def resize_and_pad(image: Image.Image, target_size: tuple[int, int], fill_color: tuple[int, int, int] = (255, 255, 255)) -> Image.Image:
    """
    Изменяет размер изображения, сохраняя пропорции, и добавляет поля до target_size.
    """
    # Создаем новое квадратное изображение с заданным цветом фона
    padded_image = Image.new("RGB", target_size, fill_color)
    
    # Уменьшаем исходное изображение так, чтобы оно вписывалось в target_size
    image.thumbnail(target_size, Image.Resampling.LANCZOS)
    
    # Вычисляем позицию для центрирования
    paste_position = (
        (target_size[0] - image.width) // 2,
        (target_size[1] - image.height) // 2
    )
    
    # Вставляем уменьшенное изображение на фон
    padded_image.paste(image, paste_position)
    
    return padded_image


# --- 2. Обновленные функции генерации ---

def generate_mesh_from_image(image_path: str, output_path: str):
    if not os.path.exists(image_path):
        print(f"Ошибка: Файл изображения не найден: {image_path}")
        return

    image = Image.open(image_path).convert("RGB")
    
    # ИСПОЛЬЗУЕМ НОВУЮ ФУНКЦИЮ ВМЕСТО ПРОСТОГО RESIZE
    processed_image = resize_and_pad(image, (512, 512))

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # СОХРАНЯЕМ РЕФЕРЕНСНОЕ ИЗОБРАЖЕНИЕ
    ref_image_path = os.path.splitext(output_path)[0] + '.png'
    processed_image.save(ref_image_path)
    print(f"Референсное изображение сохранено: {ref_image_path}")

    generated_mesh = shape_pipeline(image=processed_image, show_progress_bar=False)[0]
    generated_mesh.export(output_path)
    
    print(f"Меш из изображения сохранен: {output_path}")


def generate_mesh_from_text(prompt: str, output_path: str):
    image = image_gen_pipeline(prompt=prompt).images[0]
    
    # ИСПОЛЬЗУЕМ НОВУЮ ФУНКЦИЮ ВМЕСТО ПРОСТОГО RESIZE
    processed_image = resize_and_pad(image, (512, 512))

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # СОХРАНЯЕМ РЕФЕРЕНСНОЕ ИЗОБРАЖЕНИЕ
    ref_image_path = os.path.splitext(output_path)[0] + '.png'
    processed_image.save(ref_image_path)
    print(f"Референсное изображение сохранено: {ref_image_path}")

    generated_mesh = shape_pipeline(image=processed_image, show_progress_bar=False)[0]
    generated_mesh.export(output_path)
    
    print(f"Меш из текста '{prompt}' сохранен: {output_path}")


2025-08-02 20:55:14,808 - hy3dgen.shapgen - INFO - Try to load model from local path: /root/.cache/hy3dgen/tencent/Hunyuan3D-2.1/hunyuan3d-dit-v2-1
2025-08-02 20:55:14,815 - hy3dgen.shapgen - INFO - Loading model from /root/.cache/hy3dgen/tencent/Hunyuan3D-2.1/hunyuan3d-dit-v2-1/model.fp16.ckpt


using moe
using moe
using moe
using moe
using moe
using moe
PointCrossAttentionEncoder INFO: pc_sharpedge_size is zero


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [2]:
generate_mesh_from_image(
    image_path='../assets/d98mtp939hpc1.jpeg',
    output_path='../output/rebecca_one_piece_bikini.glb'
)


Референсное изображение сохранено: ../output/rebecca_one_piece_bikini.png


Volume Decoding: 100%|██████████████████████████████████| 7134/7134 [00:13<00:00, 513.18it/s]


Меш из изображения сохранен: ../output/rebecca_one_piece_bikini.glb


In [8]:
generate_mesh_from_text(
    prompt="(masterpiece), (best quality), game asset, a single longsword, front view, orthographic, 3d model, 3d render, hyper detailed, clean, ((white background)), ((isolated on white)), professional, studio lighting, sharp focus",
    output_path='../output/sword.glb'
)

  0%|          | 0/50 [00:00<?, ?it/s]

Референсное изображение сохранено: ../output/sword.png


Volume Decoding: 100%|██████████████████████████████████| 7134/7134 [00:13<00:00, 514.27it/s]


Меш из текста '(masterpiece, best quality, 8k), (one single sword:1.5), a single ornate fantasy sword, solo object, centered, full body shot of a sword, detailed weapon, game asset, ((isolated on a pure white background)), flat lighting, no shadows' сохранен: ../output/sword.glb


In [5]:
import torch
from diffusers import DiffusionPipeline
from hy3dshape.pipelines import Hunyuan3DDiTFlowMatchingPipeline
from PIL import Image, ImageOps
import os
import warnings
import trimesh
import numpy as np

warnings.filterwarnings("ignore")

# --- 1. Инициализация моделей ---
print("Инициализация моделей...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

shape_pipeline = Hunyuan3DDiTFlowMatchingPipeline.from_pretrained(
    'tencent/Hunyuan3D-2.1', torch_dtype=torch.float16
)

image_gen_pipeline = DiffusionPipeline.from_pretrained(
    "playgroundai/playground-v2.5-1024px-aesthetic", torch_dtype=torch.float16, variant="fp16"
).to(device)

print("Модели успешно инициализированы.")


# --- 2. Вспомогательные функции ---

def resize_and_pad(image: Image.Image, target_size: tuple[int, int], fill_color: tuple[int, int, int] = (255, 255, 255)) -> Image.Image:
    padded_image = Image.new("RGB", target_size, fill_color)
    image.thumbnail(target_size, Image.Resampling.LANCZOS)
    paste_position = ((target_size[0] - image.width) // 2, (target_size[1] - image.height) // 2)
    padded_image.paste(image, paste_position)
    return padded_image

# ФУНКЦИЯ 1: УДАЛЕНИЕ "ОСТРОВОВ"
def clean_mesh_by_largest_component(input_path: str, output_path: str) -> bool:
    try:
        print(f"Шаг 1/2: Очистка от отдельных артефактов в {input_path}")
        mesh = trimesh.load_mesh(input_path, force='mesh')
        if not isinstance(mesh, trimesh.Trimesh): return False
        
        components = mesh.split(only_watertight=False)
        
        if len(components) <= 1:
            print("Меш уже состоит из одного компонента, очистка не требуется.")
            mesh.export(output_path)
            return True
            
        print(f"Найдено компонентов: {len(components)}. Выбираем самый большой.")
        largest_component = max(components, key=lambda c: len(c.faces))
        largest_component.export(output_path)
        print(f"Очищенный high-poly меш сохранен: {output_path}")
        return True
    except Exception as e:
        print(f"!!! Ошибка на шаге 1 (очистка компонентов): {e}")
        return False

# ФУНКЦИЯ 2: УПРОЩЕНИЕ ДО LOW-POLY
def decimate_mesh(input_path: str, output_path: str, target_face_count: int) -> bool:
    try:
        print(f"Шаг 2/2: Упрощение до low-poly из {input_path}")
        mesh = trimesh.load_mesh(input_path, force='mesh')
        if not isinstance(mesh, trimesh.Trimesh): return False
        
        if len(mesh.faces) <= target_face_count:
            mesh.export(output_path)
            return True
        
        new_mesh = mesh.simplify_quadric_decimation(face_count=target_face_count)
        new_mesh.export(output_path)
        print(f"Финальный low-poly меш сохранен: {output_path}")
        return True
    except Exception as e:
        print(f"!!! Ошибка на шаге 2 (упрощение): {e}")
        return False


# --- 3. Основные конвейеры генерации ---

def process_generated_mesh(raw_path, output_prefix, low_poly_faces):
    """Упрощенный конвейер постобработки."""
    cleaned_path = f"{output_prefix}_1_cleaned_high.obj"
    final_low_poly_path = f"{output_prefix}_2_final_low.obj"

    # Шаг удаления плоскости ПОЛНОСТЬЮ УДАЛЕН
    if not clean_mesh_by_largest_component(raw_path, cleaned_path): return
    if not decimate_mesh(cleaned_path, final_low_poly_path, target_face_count=low_poly_faces): return

def generate_mesh_from_text(prompt: str, output_prefix: str, low_poly_faces: int = 1500):
    print("\n" + "="*50)
    print(f"Начало генерации из текста: '{prompt}'")
    try:
        output_dir = os.path.dirname(output_prefix)
        if not os.path.exists(output_dir): os.makedirs(output_dir)
        raw_path = f"{output_prefix}_0_raw.obj"

        print("Генерация референсного изображения...")
        image = image_gen_pipeline(prompt=prompt, num_inference_steps=25).images[0]
        processed_image = resize_and_pad(image, (512, 512))
        
        print("Генерация сырого 3D-меша...")
        generated_mesh = shape_pipeline(image=processed_image, show_progress_bar=True)[0]
        generated_mesh.export(raw_path)
        
        process_generated_mesh(raw_path, output_prefix, low_poly_faces)

    except Exception as e:
        print(f"!!! Произошла глобальная ошибка: {e}")
    print("Генерация завершена.")
    print("="*50 + "\n")

def generate_mesh_from_image(image_path: str, output_prefix: str, low_poly_faces: int = 1500):
    print("\n" + "="*50)
    print(f"Начало генерации из изображения: {image_path}")
    if not os.path.exists(image_path):
        print(f"!!! Ошибка: Файл не найден {image_path}")
        return
    try:
        output_dir = os.path.dirname(output_prefix)
        if not os.path.exists(output_dir): os.makedirs(output_dir)
        raw_path = f"{output_prefix}_0_raw.obj"

        print("Обработка референсного изображения...")
        image = Image.open(image_path).convert("RGB")
        processed_image = resize_and_pad(image, (512, 512))
        
        print("Генерация сырого 3D-меша...")
        generated_mesh = shape_pipeline(image=processed_image, show_progress_bar=True)[0]
        generated_mesh.export(raw_path)
        
        process_generated_mesh(raw_path, output_prefix, low_poly_faces)

    except Exception as e:
        print(f"!!! Произошла глобальная ошибка: {e}")
    print("Генерация завершена.")
    print("="*50 + "\n")


2025-08-03 13:14:50,873 - hy3dgen.shapgen - INFO - Try to load model from local path: /root/.cache/hy3dgen/tencent/Hunyuan3D-2.1/hunyuan3d-dit-v2-1
2025-08-03 13:14:50,880 - hy3dgen.shapgen - INFO - Loading model from /root/.cache/hy3dgen/tencent/Hunyuan3D-2.1/hunyuan3d-dit-v2-1/model.fp16.ckpt


Инициализация моделей...
Используемое устройство: cuda
using moe
using moe
using moe
using moe
using moe
using moe
PointCrossAttentionEncoder INFO: pc_sharpedge_size is zero


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Модели успешно инициализированы.


In [6]:
if __name__ == '__main__':
    # Создаем папку для результатов, если ее нет
    if not os.path.exists('output'):
        os.makedirs('output')

    # --- Пример 1: Генерация из текстового запроса ---
    # Вы можете менять prompt и количество полигонов (low_poly_faces)
    generate_mesh_from_text(
        prompt="(masterpiece), (best quality), game asset, a single longsword, front view, orthographic, 3d model, 3d render, hyper detailed, clean, ((white background)), ((isolated on white)), professional, studio lighting, sharp focus",
        output_prefix="../output/sword",
        low_poly_faces=1000  # Устанавливаем желаемое количество полигонов
    )

    # --- Пример 2: Генерация из файла ---
    # Чтобы использовать этот пример:
    # 1. Создайте файл 'my_image.png' в той же папке, что и скрипт.
    # 2. Раскомментируйте строки ниже.
    
    # image_file = '../assets/photo_2025-08-03_13-08-51.jpg'
    # if os.path.exists(image_file):
    #     generate_mesh_from_image(
    #         image_path=image_file,
    #         output_prefix="../output/rebecca_test",
    #         low_poly_faces=2000 # Другое значение для примера
    #     )
    # else:
    #     print(f"Для запуска примера 2 создайте файл с именем '{image_file}'")


Начало генерации из текста: '(masterpiece), (best quality), game asset, a single longsword, front view, orthographic, 3d model, 3d render, hyper detailed, clean, ((white background)), ((isolated on white)), professional, studio lighting, sharp focus'
Генерация референсного изображения...


  0%|          | 0/25 [00:00<?, ?it/s]

Генерация сырого 3D-меша...


Volume Decoding: 100%|████████████████████████████████████████| 7134/7134 [00:13<00:00, 510.21it/s]


Шаг 1/2: Очистка от отдельных артефактов в ../output/sword_0_raw.obj
Найдено компонентов: 5. Выбираем самый большой.
Очищенный high-poly меш сохранен: ../output/sword_1_cleaned_high.obj
Шаг 2/2: Упрощение до low-poly из ../output/sword_1_cleaned_high.obj
Финальный low-poly меш сохранен: ../output/sword_2_final_low.obj
Генерация завершена.


Начало генерации из изображения: ../assets/photo_2025-08-03_13-08-51.jpg
Обработка референсного изображения...
Генерация сырого 3D-меша...


Volume Decoding: 100%|████████████████████████████████████████| 7134/7134 [00:14<00:00, 507.66it/s]


Шаг 1/2: Очистка от отдельных артефактов в ../output/rebecca_test_0_raw.obj
Найдено компонентов: 4. Выбираем самый большой.
Очищенный high-poly меш сохранен: ../output/rebecca_test_1_cleaned_high.obj
Шаг 2/2: Упрощение до low-poly из ../output/rebecca_test_1_cleaned_high.obj
Финальный low-poly меш сохранен: ../output/rebecca_test_2_final_low.obj
Генерация завершена.



In [8]:
# -*- coding: utf-8 -*-

import torch
from diffusers import DiffusionPipeline
from hy3dshape.pipelines import Hunyuan3DDiTFlowMatchingPipeline
from PIL import Image, ImageOps
from rembg import remove
import os
import warnings
import trimesh
import numpy as np

warnings.filterwarnings("ignore")

# --- 1. Инициализация моделей ---
print("Инициализация моделей...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
shape_pipeline = Hunyuan3DDiTFlowMatchingPipeline.from_pretrained(
    'tencent/Hunyuan3D-2.1', torch_dtype=torch.float16
)
image_gen_pipeline = DiffusionPipeline.from_pretrained(
    "playgroundai/playground-v2.5-1024px-aesthetic", torch_dtype=torch.float16, variant="fp16"
).to(device)
print("Модели успешно инициализированы.")

# --- 2. Вспомогательные функции ---

def resize_and_pad(image: Image.Image, target_size: tuple[int, int], fill_color: tuple[int, int, int] = (255, 255, 255)) -> Image.Image:
    padded_image = Image.new("RGB", target_size, fill_color)
    image.thumbnail(target_size, Image.Resampling.LANCZOS)
    paste_position = ((target_size[0] - image.width) // 2, (target_size[1] - image.height) // 2)
    padded_image.paste(image, paste_position)
    return padded_image

def clean_mesh_by_largest_component(input_path: str, output_path: str) -> bool:
    try:
        mesh = trimesh.load_mesh(input_path, force='mesh')
        if not isinstance(mesh, trimesh.Trimesh): return False
        components = mesh.split(only_watertight=False)
        if len(components) <= 1:
            mesh.export(output_path)
            return True
        largest_component = max(components, key=lambda c: len(c.faces))
        largest_component.export(output_path)
        return True
    except Exception as e:
        print(f"Ошибка при очистке меша: {e}")
        return False

def decimate_mesh(input_path: str, output_path: str, target_face_count: int) -> bool:
    try:
        mesh = trimesh.load_mesh(input_path, force='mesh')
        if not isinstance(mesh, trimesh.Trimesh): return False
        if len(mesh.faces) <= target_face_count:
            mesh.export(output_path)
            return True
        new_mesh = mesh.simplify_quadric_decimation(face_count=target_face_count)
        new_mesh.export(output_path)
        return True
    except Exception as e:
        print(f"Ошибка при упрощении меша: {e}")
        return False

def process_generated_mesh(raw_path, output_prefix, low_poly_faces):
    cleaned_path = f"{output_prefix}_1_cleaned_high.obj"
    final_low_poly_path = f"{output_prefix}_2_final_low.obj"
    if not clean_mesh_by_largest_component(raw_path, cleaned_path): return
    if not decimate_mesh(cleaned_path, final_low_poly_path, target_face_count=low_poly_faces): return

# --- 3. Основной конвейер генерации ---

def generate_mesh_from_text(
    prompt: str,
    output_prefix: str,
    low_poly_faces: int = 1500,
    seed: int = 42,
    num_inference_steps: int = 50,
    guidance_scale: float = 3.5
):
    try:
        output_dir = os.path.dirname(output_prefix)
        if not os.path.exists(output_dir): os.makedirs(output_dir)
        raw_path = f"{output_prefix}_0_raw.obj"

        generator = torch.Generator(device=device).manual_seed(seed)
        
        image = image_gen_pipeline(
            prompt=prompt,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            generator=generator
        ).images[0]

        image_no_bg_rgba = remove(image)
        white_background = Image.new("RGB", image_no_bg_rgba.size, (255, 255, 255))
        white_background.paste(image_no_bg_rgba, (0, 0), image_no_bg_rgba)
        
        image_on_white = white_background
        image_on_white.save(f"{output_prefix}_reference_image.png")
        
        processed_image = resize_and_pad(image_on_white, (512, 512))
        
        generated_mesh = shape_pipeline(image=processed_image, show_progress_bar=True)[0]
        generated_mesh.export(raw_path)
        
        process_generated_mesh(raw_path, output_prefix, low_poly_faces)

    except Exception as e:
        print(f"Произошла глобальная ошибка: {e}")
    print(f"Генерация для '{output_prefix}' завершена.")


# --- 4. Точка входа и пример использования ---

if __name__ == '__main__':
    output_folder = "results_3d"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Промпт для генерации объекта на белом фоне
    generate_mesh_from_text(
        prompt="(masterpiece), (best quality), game asset, a single longsword, front view, orthographic, 3d model, 3d render, hyper detailed, clean, ((white background)), ((isolated on white)), professional, studio lighting, sharp focus",
        output_prefix="../output/sword_imp_g",
        low_poly_faces=1000  # Устанавливаем желаемое количество полигонов
    )

2025-08-03 13:28:25,637 - hy3dgen.shapgen - INFO - Try to load model from local path: /root/.cache/hy3dgen/tencent/Hunyuan3D-2.1/hunyuan3d-dit-v2-1
2025-08-03 13:28:25,643 - hy3dgen.shapgen - INFO - Loading model from /root/.cache/hy3dgen/tencent/Hunyuan3D-2.1/hunyuan3d-dit-v2-1/model.fp16.ckpt


Инициализация моделей...
using moe
using moe
using moe
using moe
using moe
using moe
PointCrossAttentionEncoder INFO: pc_sharpedge_size is zero


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Модели успешно инициализированы.


  0%|          | 0/50 [00:00<?, ?it/s]

Volume Decoding: 100%|████████████████████████████████████████| 7134/7134 [00:13<00:00, 517.31it/s]


Генерация для '../output/sword_imp_g' завершена.


In [9]:
generate_mesh_from_text(
    prompt="photorealistic 3d model of a mechanical dragon, steampunk style, single object, centered, uniform pure white background, no shadow, studio lighting, product shot, 8k",
    output_prefix="../output/dragon",
    low_poly_faces=2000  # Устанавливаем желаемое количество полигонов
)

  0%|          | 0/50 [00:00<?, ?it/s]

Volume Decoding: 100%|████████████████████████████████████████| 7134/7134 [00:13<00:00, 512.02it/s]


Генерация для '../output/dragon' завершена.
